# Essais de requêtes

## Divisions et URLs

In [10]:
# !/usr/bin/ python3
#-*- coding : utf-8 -*- 
import requests, sys, json
 
server = "https://rest.ensembl.org"
ext_div = "/info/divisions?"
 
d = requests.get(server+ext_div, headers={ "Content-Type" : "application/json"})
 
if not d.ok:
  d.raise_for_status()
  sys.exit()
 
divs = d.json() #liste avec les divisions
print(divs)



['EnsemblMetazoa', 'EnsemblFungi', 'EnsemblPlants', 'EnsemblProtists', 'EnsemblVertebrates', 'EnsemblBacteria']


## Accession
Aller chercher un numéro d'accession pour l'espèce qui servira d'entrée pour trouver la BdD ensembl appropriée

In [17]:
# !/usr/bin/ python3
#-*- coding : utf-8 -*- 
import requests, sys

def RetrieveDiv (species) :
    # Requête des infos concernant l'espèce
    server = "https://rest.ensembl.org" # Serveur Ensembl pour requête API

    ext = f"/info/assembly/{species}?" # Requête API
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"}) 

    # Retour en cas d'erreur
    if not r.ok:
        r.raise_for_status()
        sys.exit() 
        
    decoded = r.json() # Résultat API
    accession = decoded['assembly_accession'] # Récupération d'une accession d'un génome    

    # Requête des infos concernant l'accession
    ext = f"/info/genomes/assembly/{accession}?"
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"}) 

    # Retour en cas d'erreur
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    decoded = r.json() # Résultat API

    if decoded["division"][7:].lower() == "vertebrates" :
        div = "www."
    else : 
        div = decoded["division"][7:].lower()
    
    return(div)

def InfoGene (species, symbol) :
    # Serveur Ensembl pour requête API
    server = "https://rest.ensembl.org"

    # Lien vers l'espèce et symbole de gène courants
    ext = f"/lookup/symbol/{species}/{symbol}?expand=1" 
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    decoded = r.json()

    # Retour en cas d'erreur d'info de l'API
    if not r.ok:
        r.raise_for_status()
        sys.exit()

    # Construction du dictionnaire contenant les infos fomatées pour le html
    species_info = {}
    species_info["gene_symbol"] = symbol
    species_info["gene_id"] = decoded["id"]
    species_info["division"] = RetrieveDiv(species)

    for i in decoded["Transcript"] :
        if "transcript_id" not in species_info :
            species_info["transcript_id"] = i["id"]
            species_info["prot_id"] = i["Translation"]["id"]
        else : 
            species_info["transcript_id"] += "<br>\n\t\t\t" + i["id"]
            species_info["prot_id"] += "<br>\n\t\t\t" + i["Translation"]["id"]

    species_info["gene_browser"] = f"http://{species_info["division"]}.ensembl.org/{species}/Location/View?db=core;g={species_info["gene_id"]}"
    return(species_info)

with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        gene = line.split(",")[0]
        current_species = line[:-1].split(",")[1]
    div = RetrieveDiv(entries)
    url = f"http://{div}.ensembl.org/{entries}/Location/View?db=core;g=YDL168W"

print(url)




vertebrates


In [21]:
species_info = {}
with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        gene = line.split(",")[0]
        current_species = line[:-1].split(",")[1]
        species_info["gene_symbol"] = gene



escherichia_coli_gca_006381815
{'gene_symbol': 'HYBE'}


## Espèce [deprecated]

En fonction de la division
Aller chercher : name, display name, accession et division

In [10]:
# !/usr/bin/ python3
#-*- coding : utf-8 -*- 

import requests, sys, json
 
server = "https://rest.ensembl.org"
ext = "/info/species?division=EnsemblBacteria"
divisions = ['EnsemblMetazoa', 'EnsemblFungi', 'EnsemblPlants', 'EnsemblProtists', 'EnsemblVertebrates', 'EnsemblBacteria']
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
print (type(decoded))
a= decoded.keys()
print(a)
print(decoded["species"][1]["division"])

test_file = open("test_species.txt","a")
test_file.write(json.dumps(decoded,indent=2))
test_file.close()

<class 'dict'>
dict_keys(['species'])
EnsemblBacteria


## Gène
`"id": "ENST` Récupère les ID des transcrits

In [23]:
# !/usr/bin/ python3
#-*- coding : utf-8 -*- 
import requests, sys, json

# Lecture du fichier d'espèce et symboles de gènes
with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        symbol = line.split(",")[0]
        current_species = line[:-1].split(",")[1]

def InfoGene (species, symbol) :
    # Serveur Ensembl pour requête API
    server = "https://rest.ensembl.org"

    # Lien vers l'espèce et symbole de gène courants
    ext = f"/lookup/symbol/{species}/{symbol}?expand=1" 
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    decoded = r.json()

    # Retour en cas d'erreur d'info de l'API
    if not r.ok:
        r.raise_for_status()
        sys.exit()

    # Construction du dictionnaire contenant les infos fomatées pour le html
    species_info = {}
    species_info["gene_symbol"] = symbol
    species_info["gene_id"] = decoded["id"]

    for i in decoded["Transcript"] :
        if "transcript_id" not in species_info :
            species_info["transcript_id"] = i["id"]
            species_info["prot_id"] = i["Translation"]["id"]
        else : 
            species_info["transcript_id"] += "<br>\n\t\t\t" + i["id"]
            species_info["prot_id"] += "<br>\n\t\t\t" + i["Translation"]["id"]

    return(species_info)

print(InfoGene("homo_sapiens","RAD51"))

{'gene_symbol': 'RAD51', 'gene_id': 'ENSG00000051180', 'transcript_id': 'ENST00000527860<br>\n\t\t\tENST00000423169<br>\n\t\t\tENST00000557850<br>\n\t\t\tENST00000382643<br>\n\t\t\tENST00000267868<br>\n\t\t\tENST00000645673<br>\n\t\t\tENST00000525066<br>\n\t\t\tENST00000526763<br>\n\t\t\tENST00000532743<br>\n\t\t\tENST00000531277<br>\n\t\t\tENST00000533741', 'prot_id': 'ENSP00000432759<br>\n\t\t\tENSP00000406602<br>\n\t\t\tENSP00000454176<br>\n\t\t\tENSP00000372088<br>\n\t\t\tENSP00000267868<br>\n\t\t\tENSP00000493712<br>\n\t\t\tENSP00000431864<br>\n\t\t\tENSP00000431897<br>\n\t\t\tENSP00000433924<br>\n\t\t\tENSP00000436512<br>\n\t\t\tENSP00000433053'}


## Création HTML
écrire structure et modifier html

In [22]:
dic = {'homo_sapiens': [{'symbol': 'RAD51', 'gene_id': 'ENSG00000051180', 'transcript_id': 'ENST00000527860<br>\n\t\t\tENST00000423169<br>\n\t\t\tENST00000557850<br>\n\t\t\tENST00000382643<br>\n\t\t\tENST00000267868<br>\n\t\t\tENST00000645673<br>\n\t\t\tENST00000525066<br>\n\t\t\tENST00000526763<br>\n\t\t\tENST00000532743<br>\n\t\t\tENST00000531277<br>\n\t\t\tENST00000533741', 'prot_id': 'ENSP00000432759<br>\n\t\t\tENSP00000406602<br>\n\t\t\tENSP00000454176<br>\n\t\t\tENSP00000372088<br>\n\t\t\tENSP00000267868<br>\n\t\t\tENSP00000493712<br>\n\t\t\tENSP00000431864<br>\n\t\t\tENSP00000431897<br>\n\t\t\tENSP00000433924<br>\n\t\t\tENSP00000436512<br>\n\t\t\tENSP00000433053'}], 'arabidopsis_thaliana': [{'symbol': 'PHYB', 'gene_id': 'AT2G18790', 'transcript_id': 'AT2G18790.2<br>\n\t\t\tAT2G18790.1', 'prot_id': 'AT2G18790.2<br>\n\t\t\tAT2G18790.1'}], 'saccharomyces_cerevisiae': [{'symbol': 'SFA1', 'gene_id': 'YDL168W', 'transcript_id': 'YDL168W_mRNA', 'prot_id': 'YDL168W'}], 'Leishmania_major': [{'symbol': 'ACP', 'gene_id': 'LMJF_27_0290', 'transcript_id': 'CBZ12089', 'prot_id': 'CBZ12089'}], 'escherichia_coli_gca_006381815': [{'symbol': 'HYBE', 'gene_id': 'ENSB:fqjn3NbmMv9mSox', 'transcript_id': 'ENSB:fqjn3NbmMv9mSox', 'prot_id': 'ENSB:fqjn3NbmMv9mSox'}]}
html = open("EMBL.html","w")

## HTML Head
head_html = """
<!DOCTYPE HTML>
<html lang="fr-FR">
	<head>
		<title> Formulaire javascript 1 </title>
		<meta charset=UTF-8/>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <!-- CSS -->
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.20/css/jquery.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/fixedcolumns/3.3.0/css/fixedColumns.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/buttons/1.7.1/css/buttons.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/fixedheader/3.1.9/css/fixedHeader.dataTables.min.css">
    
        <!-- JavaScript -->
        <script type="text/javascript" language="javascript" src="https://code.jquery.com/jquery-3.7.1.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/1.10.20/js/jquery.dataTables.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/fixedcolumns/3.3.0/js/dataTables.fixedColumns.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/buttons/1.7.1/js/dataTables.buttons.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/buttons/1.7.1/js/buttons.colVis.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/fixedheader/3.1.9/js/dataTables.fixedHeader.min.js"></script>
        <script type="text/javascript" language="javascript" src="https://cdn.datatables.net/rowgroup/1.1.2/js/dataTables.rowGroup.min.js"></script>
        
        <!-- Script Supplémentaire-->
        <script type="text/javascript" class="init">
            $(document).ready(function() {
                const table = $('#gene').DataTable({
                    "scrollY": 700,
                    "scrollX": true,
                    lengthMenu: [
                        [5, 10, 25, 50, -1],
                        [5, 10, 25, 50, 'All']
                    ],
                    fixedColumns: {
                        leftColumns: 2
                    },
                    fixedHeader: true,
                    dom: 'Blfrtip',
                    buttons: [
                        'colvis'
                    ]
                });
            });
            </script>
    </head>
    <body>
        <h1>Resultat du scripting d'aggrégation automatique des annotations</h1>

        <table id="gene" class="display nowrap cell-border" style="width:100%">
            <thead>
                <tr>
                    <th><div class=header_1>Species</div></th>
                    <th>Gene Symbol</th>
                    <th>Gene - EMBL ID</th>
                    <th>Protein - EMBL ID</th>
                    <th>Transcript - EMBL ID</th>
                </tr>
            </thead>
            <tbody>
"""

## HTML Body
body_html =""
for esp in dic :
    champ = dic[esp][0]
    body_html += f"""
    <tr>                    
        <td><div class=header_1>
            {esp}
        </div></td>
        <td>
            {champ["symbol"]}
        </td>
        <td>
            {champ["gene_id"]}
        </td>
        <td>
            {champ["prot_id"]}
        </td>
        <td>
            {champ["transcript_id"]}
        </td>
    </tr>"""
tail_html = f""" 
            </tbody>
        </table>
    </body>
</html>
"""

html = open("EMBL.html","w")
html.write(head_html[1:]+body_html+tail_html)
html.close()


## Sauvegarde
Y'a des bouts de code utiles  
```python  
decoded = (json.dumps(decoded,indent=2))  
```